In [1]:
import pandas as pd
from loguru import logger

from pet_products_scraper import (
    PetProductsETL,
    ZooplusETL,
    PetsAtHomeETL,
    JollyesETL,
    LilysKitchenETL,
    BitibaETL,
    PetSupermarketETL,
    PetPlanetETL,
    PurinaETL,
    DirectVetETL,
    FishKeeperETL,
    PetDrugsOnlineETL,
    ViovetETL,
    PetShopETL,
    VetShopETL,
    VetUKETL,
    BurnsPetETL,
    AsdaETL,
    TheRangeETL,
    OcadoETL,
    HarringtonsETL,
    BernPetFoodsETL,
    PetsCornerETL,
    OrijenETL,
    ThePetExpressETL,
    PetShopOnlineETL,
    TaylorPetFoodsETL,
    TheNaturalPetStoreETL,
    HealthyPetStoreETL,
    FarmAndPetPlaceETL,
    NaturesMenuETL,
)

factory = {
    # "Zooplus": ZooplusETL(),  
    "PetsAtHome": PetsAtHomeETL(),  
    "Jollyes": JollyesETL(),  
    "LilysKitchen": LilysKitchenETL(),  
    "Bitiba": BitibaETL(),
    "PetSupermarket": PetSupermarketETL(),  
    "PetPlanet": PetPlanetETL(),  
    "Purina": PurinaETL(),  
    "DirectVet": DirectVetETL(),  
    "FishKeeper": FishKeeperETL(),  
    "PetDrugsOnline": PetDrugsOnlineETL(),  
    "Viovet": ViovetETL(),  
    "PetShop": PetShopETL(),  
    "VetShop": VetShopETL(),  
    "VetUK": VetUKETL(),  
    "BurnsPet": BurnsPetETL(),  
    "ASDAGroceries": AsdaETL(),  
    # "TheRange": TheRangeETL(),
    "Ocado": OcadoETL(),  
    "Harringtons": HarringtonsETL(),  
    "BernPetFoods": BernPetFoodsETL(),  
    "PetsCorner": PetsCornerETL(),  
    "Orijen": OrijenETL(),  
    "ThePetExpress": ThePetExpressETL(),  
    "PetShopOnline": PetShopOnlineETL(),  
    "TaylorPetFoods": TaylorPetFoodsETL(),  
    "TheNaturalPetStore": TheNaturalPetStoreETL(),  
    "HealthyPetStore": HealthyPetStoreETL(),  
    "FarmAndPetPlace": FarmAndPetPlaceETL(),  
    "NaturesMenu": NaturesMenuETL(),  
}

def run_etl(shop: str):

    if shop in factory:
        return factory[shop]
    else:
        raise ValueError(
            f"Shop {shop} is not supported. Please pass a valid shop.")

2025-04-29 16:31:41.809 | INFO     | pet_products_scraper._pet_products_etl:extract_from_url:59 - Successfully extracted data from https://www.direct-vet.co.uk 200
2025-04-29 16:31:41.810 | INFO     | pet_products_scraper._pet_products_etl:extract_from_url:64 - Sleeping for 0.09734865652639413 seconds...
2025-04-29 16:31:43.881 | INFO     | pet_products_scraper._pet_products_etl:extract_from_url:59 - Successfully extracted data from https://www.direct-vet.co.uk 200
2025-04-29 16:31:43.882 | INFO     | pet_products_scraper._pet_products_etl:extract_from_url:64 - Sleeping for 1.211572753762822 seconds...


In [2]:
d = pd.read_csv('./csv/pet_product_variant_urls.csv')
d['full_url'] = d['base_url'].str[:-1] + d['url']
d['full_url'] = d['full_url'].str.replace(d['base_url'].str[:-1], '')

In [3]:
company_list = d['shop_name'].value_counts().reset_index()['shop_name'].to_list()
valid_companies = [company for company in company_list if company in factory.keys()]

valid_companies

['Viovet',
 'PetShop',
 'Bitiba',
 'ThePetExpress',
 'Ocado',
 'FarmAndPetPlace',
 'PetSupermarket',
 'PetsCorner',
 'PetsAtHome',
 'PetDrugsOnline',
 'VetUK',
 'Jollyes',
 'HealthyPetStore',
 'TheNaturalPetStore',
 'PetPlanet',
 'PetShopOnline',
 'DirectVet',
 'ASDAGroceries',
 'TaylorPetFoods',
 'LilysKitchen',
 'NaturesMenu',
 'BernPetFoods',
 'Harringtons',
 'Orijen']

In [4]:
def extract(companies):
    for c in companies:
        sample_df = d[d['shop_name'] == c]
        scrape_link = sample_df['full_url'].value_counts().reset_index()['full_url'].to_list()
        scraper = run_etl(c)
        scrape_payload = []
        for link in scrape_link:
            scrape_df = scraper.image_scrape_product(link)
            if scrape_df is not None:
                scrape_payload.append(scrape_df)
            else:
                raise ValueError('Problem with scraper . . ')

        transform(c, scrape_payload)


def transform(company, df):
    df_scrape = pd.DataFrame(df)

    df = d[d['shop_name'] == company]

    df_merge = df.merge(df_scrape, how="inner", left_on="full_url", right_on="url")
    load(df_merge[['id', 'shop_name', 'base_url', 'image_urls']], company)


def load(df, company):
    df.to_csv(f'./csv/{company}.csv', index=False)



extract(['PetSupermarket'])

KeyError: 'url'